### Import modules

In [2]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

In [59]:
def return_subfolders(parent):
    '''list of all subfolders'''
    for folder in os.listdir(parent):
        # exclude non-folders and hidden files
        if not folder.startswith('.') and os.path.isdir(os.path.join(path_train, folder)) : 
            yield folder

In [91]:
path_train = "data/train_another"
path_val = "data/validation_another"
path_test = "data/test"
x_train=[]
x_val = []
x_test = []


for samples_array, folder_path in ((x_train, path_train), (x_val, path_val), (x_test, path_test)):
    for folder in return_subfolders(folder_path):
        sub_path = folder_path + "/" + folder
        for i, img in enumerate(os.listdir(sub_path)):
            image_path = sub_path + "/" +img
            img_arr=cv2.imread(image_path)
            img_arr=cv2.resize(img_arr,(128,128))
            samples_array.append(img_arr)
            # print(f'image {i} processed')

print(f"{len(x_train)} training samples")
print(f"{len(x_val)} validation samples")
print(f"{len(x_test)} testing samples")

10000 training samples
2000 validation samples
2000 testing samples
